In [1]:
import dask_awkward as dak
import awkward as ak
import pandas as pd

from distributed import Client
import glob
import os

client =  Client(n_workers=40,  threads_per_worker=1, processes=True, memory_limit='8 GiB')

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43859 instead
  warnings.warn(


In [2]:
inFile = "/eos/purdue/store/user/rasharma/customNanoAOD_Gautschi_v2/UL2017/VBFHToMuMu_M125_TuneCP5_withDipoleRecoil_13TeV-powheg-pythia8/8996A2F7-0207-6D4B-B4AE-FE0C3C4BFEC1_NanoAOD.root"

In [3]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

NanoAODSchema.warn_missing_crossrefs = False

events = NanoEventsFactory.from_root(
    {inFile: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "VBFHToMuMu"},
).events()
events = events[:100]

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

In [4]:
muons = events.Muon
nMuons = ak.num(muons)

In [5]:
jets = events.Jet
jet_mask = (jets.pt > 30) & (abs(jets.eta) < 4.7)
selected_jets = jets[jet_mask]

In [6]:
dr = selected_jets.metric_table(muons)

jet_mu_mask = ak.all(dr > 0.4, axis=2)
selected_jets = selected_jets[jet_mu_mask]
nJets = ak.num(selected_jets)

In [7]:
event_mask = (nMuons == 2) & (nJets >= 2)  
events        = events[event_mask]

In [8]:
events[:10].Muon.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

<Array [[63.7, 61.9], [...], ..., [62.7, 56.1]] type='10 * var * float32[pa...'>

In [9]:
events[:10].Jet.pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [[68.1, 67.1, ..., 16.4, 15.5], ...] type='10 * var * float32[parame...'>

In [10]:
selected_jets = selected_jets[event_mask]
selected_jets[:10].pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

<Array [[56.7, 51.1], [...], ..., [45.5, 43.7]] type='10 * var * float32[pa...'>

In [11]:
muons = muons[event_mask]
muons[:10].pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

<Array [[63.7, 61.9], [...], ..., [62.7, 56.1]] type='10 * var * float32[pa...'>

In [12]:
mu1, mu2 = muons[:, 0], muons[:, 1]

In [13]:
njets = ak.num(selected_jets, axis=1)
padded_jets = ak.pad_none(selected_jets, 2)
jet1 = padded_jets[:,0]
jet2 = padded_jets[:,1]

In [15]:
new_events = events
saj = new_events.SoftActivityJet

dR_m1 = saj.delta_r(mu1)
dR_m2 = saj.delta_r(mu2)
dR_j1 = saj.delta_r(jet1)
dR_j2 = saj.delta_r(jet2)
dR_m1_filter = ak.fill_none((dR_m1 > 0.4), value=True, axis=None)
dR_m2_filter = ak.fill_none((dR_m2 > 0.4), value=True, axis=None)
dR_j1_filter = ak.fill_none((dR_j1 > 0.4), value=True, axis=None)
dR_j2_filter = ak.fill_none((dR_j2 > 0.4), value=True, axis=None)
print(f"jets dR_m1_filter: {dR_m1_filter}")
print(f"jets dR_m2_filter: {dR_m2_filter}")
print(f"jets dR_j1_filter: {dR_j1_filter}")
print(f"jets dR_j2_filter: {dR_j2_filter}")

jets dR_m1_filter: dask.awkward<fill-none, npartitions=1>
jets dR_m2_filter: dask.awkward<fill-none, npartitions=1>
jets dR_j1_filter: dask.awkward<fill-none, npartitions=1>
jets dR_j2_filter: dask.awkward<fill-none, npartitions=1>


In [17]:
saj[:10].pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [[64.4, 62.3, ..., 3.69, 2.92], ...] type='10 * var * float32[parame...'>

In [19]:
dR_m1[:10].compute()

<Array [[0.00251, 2.63, 2.44, ..., 1.59, 1.91], ...] type='10 * var * float32'>

In [23]:
dR_m1_filter[:10].compute()

<Array [[False, True, True, True, True, True], ...] type='10 * var * bool'>

In [20]:
good_saj = dR_m1_filter & dR_m2_filter & dR_j1_filter & dR_j2_filter
softjet_cleaning = (good_saj) & (saj.pt > 2)
saj_of_interest = saj[softjet_cleaning]
ht_corrected = ak.sum(saj_of_interest.pt, axis=1)
corrected_njets = ak.num(saj_of_interest, axis=1)

In [28]:
saj[:10].pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [[64.4, 62.3, ..., 3.69, 2.92], ...] type='10 * var * float32[parame...'>

In [29]:
saj_of_interest[:10].pt.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called),

<Array [[7.82, 5.38, 3.69, 2.92], ..., [...]] type='10 * var * float32[para...'>

In [21]:
ht_corrected.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [19.8, 0, 7.68, 8.46, ..., 14.9, 7.41, 2.36, 10.2] type='33 * float32'>

In [22]:
corrected_njets.compute()

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


<Array [4, 0, 3, 3, 3, 3, 1, 2, ..., 1, 2, 2, 3, 3, 2, 1, 3] type='33 * int64'>